In [4]:
from typing import Dict, Optional, List
import numpy as np

class HMMState:
    def __init__(self, mean: List[np.ndarray], covariance: List[np.ndarray], transition: Dict["HMMState", float], label: Optional[int] = None, parent: Optional["HMMState"] = None):
        self.mean = mean
        """n_gaussians of mean vectors."""
        self.covariance = covariance
        """n_gaussians of diagonal of covariance matrix."""
        self.transition = transition
        """Transition probability to other HMMState instances."""
        self.label = label
        """The digit associated with the state. `None` if the state is the first state."""
        self.parent = parent
        """The state is the first state if the `parent` is `None`."""

    @classmethod
    def root(cls):
        """Creates a root HMMState with default parameters."""
        return cls(mean=[], covariance=[], transition={}, label=None)

    def __hash__(self) -> int:
        """Enables HMMState instances to be used as dictionary keys or in sets."""
        return id(self)

    def add_transition(self, state: "HMMState", probability: float):
        """Adds or updates a transition probability to another state."""
        self.transition[state] = probability

    def get_transition_prob(self, state: "HMMState") -> float:
        """Retrieves the transition probability to another state, if defined."""
        return self.transition.get(state, 0.0)
    
    def get_emission_prob(self, observation: np.ndarray) -> float:
        """Calculate the emission probability of an observation for this state,
        considering the state might represent multiple Gaussians."""
        total_prob = 0.0
        n_gaussians = len(self.mean)
        
        for i in range(n_gaussians):
            mean = self.mean[i]
            covariance = self.covariance[i]
            k = mean.shape[0]
            covariance_det = np.prod(covariance)
            covariance_inv = 1 / covariance
            diff = observation - mean
            exponent = -0.5 * np.sum((diff ** 2) * covariance_inv)
            coefficient = 1 / np.sqrt((2 * np.pi) ** k * covariance_det)
            total_prob += coefficient * np.exp(exponent)
        
        # Assuming equal weight for each Gaussian component
        return total_prob / n_gaussians if n_gaussians > 0 else 0

# Example of usage
if __name__ == "__main__":
    # Creating root state
    root_state = HMMState.root()
    
    # Creating another state with example data
    mean_example = [np.array([0.0, 1.0])]
    covariance_example = [np.array([1.0, 1.0])]
    state_example = HMMState(mean=mean_example, covariance=covariance_example, transition={}, label=1)
    
    # Adding a transition from root to example state
    root_state.add_transition(state_example, 0.5)

    print(root_state.get_transition_prob(state_example))  # Example of getting a transition probability


0.5


In [5]:
from typing import List, Tuple, Dict
import numpy as np

class HMM:
    def __init__(self):
        self.states: List[HMMState] = []
        self.observations: List[np.ndarray] = []
        self.state_index: Dict[HMMState, int] = {}

    def add_state(self, state: HMMState):
        """Adds a state to the HMM."""
        self.states.append(state)
        self.state_index[state] = len(self.states) - 1

    def set_observations(self, observations: List[np.ndarray]):
        """Sets the sequence of observations for the HMM."""
        self.observations = observations

    def viterbi(self, observations):
        """
        Viterbi Algorithm to find the most probable state sequence.
        Args:
            observations (List[np.ndarray]): A list of observation vectors.
        Returns:
            Tuple[List[int], float]: The most likely state sequence and its probability.
        """
        n_states = len(self.states)
        n_observations = len(observations)
        
        # Initialize matrices
        dp = np.zeros((n_states, n_observations))  # Probability matrix
        backpointer = np.zeros((n_states, n_observations), dtype=int)  # Backpointer matrix
        
        # Initial probabilities
        for s in range(n_states):
            dp[s, 0] = self.states[s].get_emission_prob(observations[0]) * (1.0 / n_states)
        
        # Viterbi algorithm
        for t in range(1, n_observations):
            for s in range(n_states):
                (prob, state) = max(
                    (dp[k, t - 1] * self.states[k].get_transition_prob(self.states[s]) * self.states[s].get_emission_prob(observations[t]), k)
                    for k in range(n_states)
                )
                dp[s, t] = prob
                backpointer[s, t] = state
        
        # Reconstruct the state sequence
        last_state = np.argmax(dp[:, -1])
        path = [last_state]
        for t in range(n_observations - 1, 0, -1):
            last_state = backpointer[last_state, t]
            path.insert(0, last_state)
        
        # Convert state indexes to labels
        best_path = [self.states[i].label for i in path]
        max_prob = np.max(dp[:, -1])
        
        return best_path, max_prob
    
    def train(self, training_data):
        """
        Train the HMM using the provided training data.
        Args:
            training_data (List[Tuple[np.ndarray, int]]): A list of tuples where the first element is an observation vector and the second is the state label.
        """
        # Simplified training process: Calculate mean and covariance for each state based on labeled data.
        for label in range(10):
            # Extract the observations for this label
            observations = [obs for obs, lbl in training_data if lbl == label]
            if not observations:
                continue
            
            # Convert to numpy array for easier manipulation
            observations = np.array(observations)
            
            # Calculate the mean and covariance for the observations of this label
            mean = np.mean(observations, axis=0)
            covariance = np.var(observations, axis=0)
            
            # Set the mean and covariance for the corresponding state
            self.states[label].mean = [mean]
            self.states[label].covariance = [covariance]

        # Assume we have uniform transition probabilities and set them accordingly
        transition_probability = 1.0 / (len(self.states) - 1)
        for state in self.states:
            for next_state in self.states:
                if state != next_state:
                    state.add_transition(next_state, transition_probability)





In [6]:
digit_states = {i: HMMState.root() for i in range(10)}

for i in range(10):
    for j in range(10):
        if i != j: 
            digit_states[i].add_transition(digit_states[j], 0.1)

silence_state = HMMState.root()
for state in digit_states.values():
    state.add_transition(silence_state, 0.05) 
    silence_state.add_transition(state, 0.05)

hmm_model = HMM()
for state in digit_states.values():
    hmm_model.add_state(state)
hmm_model.add_state(silence_state)

